In [ ]:
%pip install mlxtend --upgrade

In [ ]:
!pip install apyori
!pip install -U scikit-mine

# Qiestion 1
Test Drive ACLOSE algorithm to mine closed frequent patterns on a sample dataset of your choice. Test the same on a FIMI benchmark dataset which you have used for Apriori/FP-growth implementations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from apyori import apriori
from skmine.datasets.fimi import fetch_mushroom
import time

# Load the dataset
data = fetch_mushroom()
data.head()



def closed_itemset(min_sup, min_conf):
  frequent_itemsets = list(apriori(data, min_support=min_sup, min_confidence=min_conf))

  closed_freq_sets = []
  for i, itemset in enumerate(frequent_itemsets):
    antecedent = itemset.items
    support = itemset.support

    is_closed = True
    for j, other_itemset in enumerate(frequent_itemsets):
      if j != i:
        other_antecedent = other_itemset.items
        other_support = other_itemset.support
        
        if set(antecedent).issubset(set(other_antecedent)) and support==other_support:
          is_closed = False
          break

    if is_closed:
      closed_freq_sets.append((antecedent, support))

  return closed_freq_sets

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
start = time.process_time()
result = closed_itemset(0.5, 0.8)
end = time.process_time()

# Print the results
print("\nTime Taken:",end-start)

print("\n\nClosed itemsets:")
result

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Time Taken: 0.95025008799999


Closed itemsets:


[(frozenset({85}), 1.0),
 (frozenset({2, 85}), 0.517971442639094),
 (frozenset({24, 85}), 0.5844411619891678),
 (frozenset({34, 85}), 0.9741506646971935),
 (frozenset({36, 85}), 0.8385032003938946),
 (frozenset({39, 85}), 0.690792712949286),
 (frozenset({59, 85}), 0.637124569177745),
 (frozenset({63, 85}), 0.6075824716888233),
 (frozenset({85, 86}), 0.9753815854258986),
 (frozenset({85, 90}), 0.9217134416543574),
 (frozenset({24, 34, 85}), 0.5585918266863614),
 (frozenset({24, 85, 86}), 0.5598227474150664),
 (frozenset({24, 85, 90}), 0.5425898572131955),
 (frozenset({34, 85, 86}), 0.9731659281142294),
 (frozenset({34, 85, 90}), 0.8980797636632201),
 (frozenset({36, 39, 85}), 0.5608074839980305),
 (frozenset({36, 59, 85}), 0.5445593303791236),
 (frozenset({36, 63, 85}), 0.5150172328902018),
 (frozenset({36, 85, 86}), 0.8148695224027572),
 (frozenset({36, 85, 90}), 0.7956671590349581),
 (frozenset({39, 85, 86}), 0.6671590349581487),
 (frozenset({39, 85, 90}), 0.6125061546036435),
 (froze

# Question 2
Test Drive Pincer search to mine maximal frequent patterns on a sample dataset of your choice. Test the same on a FIMI benchmark dataset which you have used for Apriori/FP-growth implementations.

In [3]:
from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))



def mfcs_gen(sk,mfcs):
  mfcs = mfcs.copy()

  for infrequent_itemset in sk:
    for mfcs_itemset in mfcs.copy():
      if all(_item in mfcs_itemset for _item in infrequent_itemset):  # If infrequent itemset is a subset of mfcs itemset
        mfcs.remove(mfcs_itemset)

        for item in infrequent_itemset:
          updated_mfcs_itemset = mfcs_itemset.copy()
          updated_mfcs_itemset.remove(item)
          if not any(all(item in _mfcs_itemset for item in updated_mfcs_itemset) for _mfcs_itemset in mfcs):
            mfcs.append(updated_mfcs_itemset)


  return mfcs





def gen_next_ck(ck,k):
  num_freq=len(ck)
  newck=[]
  for i in range(num_freq):
    j=i+1
    while((j<num_freq) and (ck[i][:k-1]==ck[j][:k-1])):
      new_itemset=ck[i][:k-1]+[ck[i][k-1]]+[ck[j][k-1]]
      insert_in_new=False
      if k==1:
        insert_in_new=True
      elif k==2 and (new_itemset[-2:] in ck):
        insert_in_new=True
      else:
        for a in combinations(ck[:-2],k-2):
          if(list(a)+ck[-2:] not in ck):
            insert_in_new=False
      if insert_in_new:
        newck.append(new_itemset)
      j+=1

  return newck





def mfs_prune(old_ck,curr_mfs):
  new_ck=old_ck.copy()
  for c in old_ck.copy():
    for itemset in curr_mfs:
      if set(c).issubset(set(itemset)):
        new_ck.remove(c)

  return new_ck





def mfcs_prune(old_ckplus1,curr_mfcs):
  new_ckplus1=[]
  for c in old_ckplus1:
    for itemset in curr_mfcs:
      if set(c).issubset(set(itemset)):
        new_ckplus1.append(c)

  return new_ckplus1







def pincerSearch(txn,min_supp):
  items = set()
  for transaction in txn:
    items.update(transaction)
  level_k=1
  cand_freq_itemsets=[[item] for item in items]
  level_freq_itemsets=[]
  level_infreq_itemsets=[]

  mfcs=[items.copy()]
  mfs=[]

  print(f"MFCS={mfcs}\n")
  print(f"MFS={mfs}\n")

  while len(cand_freq_itemsets)!=0:
    print(f"Level {level_k}")
    print(f"C{level_k} = {cand_freq_itemsets}")

    cand_freq_itemsets_cnt=[0]*len(cand_freq_itemsets)
    mfcs_itemsets_cnt=[0]*len(mfcs)

    # step 1- read txn from db and get support for ck and mfcs
    for each in txn:
      for i,itemset in enumerate(cand_freq_itemsets):
        if set(itemset).issubset(each):
          cand_freq_itemsets_cnt[i]+=1
      for i,itemset in enumerate(mfcs):
        if set(itemset).issubset(each):
          mfcs_itemsets_cnt[i]+=1

    for itemset,supp in zip(cand_freq_itemsets,cand_freq_itemsets_cnt):
      print(f"{itemset} - {supp}")
    print('\n')
    for itemset,supp in zip(mfcs,mfcs_itemsets_cnt):
      print(f"{itemset} - {supp}")
    print('\n')

    # step 2 - add freq itemsets from mfcs to mfs
    for itemset,supp in zip(mfcs,mfcs_itemsets_cnt):
      if (itemset not in mfs) and supp>=min_supp:
        mfs.append(itemset)

    print(f"MFS - {mfs}")
    level_freq_itemsets=[]
    level_infreq_itemsets=[]
    # step 3 - infreq itemsets in ck makes sk
    for itemset,supp in zip(cand_freq_itemsets,cand_freq_itemsets_cnt):
      if supp>=min_supp:
        level_freq_itemsets.append(itemset)
      if supp<min_supp:
        level_infreq_itemsets.append(itemset)

    print(f"L{level_k} - {level_freq_itemsets}")
    print(f"S{level_k} - {level_infreq_itemsets}")

    # step 4 - mfcs-gen if sk is non empty
    mfcs=mfcs_gen(level_infreq_itemsets,mfcs)
    print(f"MFCS - {mfcs}")

    # step 5 - pruning cand using mfs
    print(f"C{level_k} was - {level_freq_itemsets}")
    level_freq_itemsets=mfs_prune(level_freq_itemsets,mfs)
    print(f"After pruning,L{level_k} - {level_freq_itemsets}")

    # step 6 - gen next ck from old ck
    cand_freq_itemsets=gen_next_ck(cand_freq_itemsets,level_k)

    # step 7 - prune new ck with mfcs
    cand_freq_itemsets=mfcs_prune(cand_freq_itemsets,mfcs)

    level_k+=1

  return mfs





transactions = [{1, 5, 6, 8},
                {2, 4, 8},
                {4, 5, 7},
                {2, 3},
                {5, 6, 7},
                {2, 3, 4},
                {2, 6, 7, 9},
                {5},
                {8},
                {3, 5, 7},
                {3, 5, 7},
                {5, 6, 8},
                {2, 4, 6, 7},
                {1, 3, 5, 7},
                {2, 3, 9},]


min_support_count = 3
MFS = pincerSearch(transactions, min_support_count)
print("MFS = {}".format(MFS))


<IPython.core.display.Javascript object>

MFCS=[{1, 2, 3, 4, 5, 6, 7, 8, 9}]

MFS=[]

Level 1
C1 = [[1], [2], [3], [4], [5], [6], [7], [8], [9]]
[1] - 2
[2] - 6
[3] - 6
[4] - 4
[5] - 8
[6] - 5
[7] - 7
[8] - 4
[9] - 2


{1, 2, 3, 4, 5, 6, 7, 8, 9} - 0


MFS - []
L1 - [[2], [3], [4], [5], [6], [7], [8]]
S1 - [[1], [9]]
MFCS - [{2, 3, 4, 5, 6, 7, 8}]
C1 was - [[2], [3], [4], [5], [6], [7], [8]]
After pruning,L1 - [[2], [3], [4], [5], [6], [7], [8]]
Level 2
C2 = [[2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [4, 5], [4, 6], [4, 7], [4, 8], [5, 6], [5, 7], [5, 8], [6, 7], [6, 8], [7, 8]]
[2, 3] - 3
[2, 4] - 3
[2, 5] - 0
[2, 6] - 2
[2, 7] - 2
[2, 8] - 1
[3, 4] - 1
[3, 5] - 3
[3, 6] - 0
[3, 7] - 3
[3, 8] - 0
[4, 5] - 1
[4, 6] - 1
[4, 7] - 2
[4, 8] - 1
[5, 6] - 3
[5, 7] - 5
[5, 8] - 2
[6, 7] - 3
[6, 8] - 2
[7, 8] - 0


{2, 3, 4, 5, 6, 7, 8} - 0


MFS - []
L2 - [[2, 3], [2, 4], [3, 5], [3, 7], [5, 6], [5, 7], [6, 7]]
S2 - [[2, 5], [2, 6], [2, 7], [2, 8], [3, 4], [3, 6], [3, 8], [4, 5], [4, 6], 